# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import os
import glob

# Donwload databases

*   Using Google Colab the files are saved into the user personal directory
*   if /resources/data does not exists it will be added

In [5]:
!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv -P /resources/data
!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv -P /resources/data

--2021-03-14 20:16:17--  http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv
Resolving prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com (prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com)... 52.218.41.212
Connecting to prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com (prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com)|52.218.41.212|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv [following]
--2021-03-14 20:16:18--  http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv
Resolving prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com (prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com)... 52.218.53.196
Connecting to prod2.publicdata.landregistry.gov.uk.s3-websit

# File to DataFrame

A list is compiled containing all the dataframes created



In [6]:
path = r'/resources/data'
all_files = glob.glob(path + "/*.csv")

all_dfs = []

for filename in all_files:
  pd_name = 'ppd_year_' + filename.split('.')[0][-4:]
  print(pd_name)
  pd_name = pd.read_csv(filename, index_col=None, header=0)
  all_dfs.append(pd_name)

ppd_year_2019
ppd_year_2020


# Adding column name to the respective columns 

In [7]:
for df in all_dfs:
    df.columns = ['Tuid', 'Price', 'DateOfTrasfer', 'Postcode','Prop_Type', 'Old/New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town/City', 'District', 'County', 'PPD', 'Record Status']

# Price Paid DataFrame is concateneted from the list of the dataframes

In [8]:
price_paid_data = pd.concat(all_dfs).reset_index(drop=True)

In [9]:
price_paid_data.head(10)

,Tuid,Price,DateOfTrasfer,Postcode,Prop_Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPD,Record Status
0,{8F1B26BD-60CB-53DB-E053-6C04A8C03649},246995,2019-03-29 00:00,TS15 9ZH,D,Y,F,16,NaN,GRESLEY CLOSE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
1,{8F1B26BD-60CC-53DB-E053-6C04A8C03649},244950,2019-05-17 00:00,TS18 2FN,T,Y,F,13,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
2,{8F1B26BD-60CD-53DB-E053-6C04A8C03649},139950,2019-05-31 00:00,TS18 2FN,S,Y,F,40,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
3,{8F1B26BD-60CE-53DB-E053-6C04A8C03649},271995,2019-05-31 00:00,TS15 9FD,D,Y,F,27,NaN,MALLARD DRIVE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
4,{8F1B26BD-60CF-53DB-E053-6C04A8C03649},84450,2019-04-26 00:00,TS18 2FD,T,Y,F,117,NaN,DEEPDALE AVENUE,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
5,{8F1B26BD-60D0-53DB-E053-6C04A8C03649},354950,2019-04-29 00:00,TS17 5JS,D,Y,F,6,NaN,PAGAN DRIVE,INGLEBY BARWICK,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
6,{8F1B26BD-60D1-53DB-E053-6C04A8C03649},279950,2019-05-31 00:00,TS17 5JP,D,Y,F,1,NaN,DUBRIS CLOSE,INGLEBY BARWICK,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
7,{8F1B26BD-60D5-53DB-E053-6C04A8C03649},115000,2019-05-03 00:00,TS26 0NW,F,Y,L,HARTFIELDS MANOR,APARTMENT 317,HARTFIELDS,NaN,HARTLEPOOL,HARTLEPOOL,HARTLEPOOL,A,A
8,{8F1B26BD-60D6-53DB-E053-6C04A8C03649},145950,2019-05-31 00:00,TS8 9GG,T,Y,F,3,NaN,STUTEVILLE ROW,HEMLINGTON,MIDDLESBROUGH,MIDDLESBROUGH,MIDDLESBROUGH,A,A
9,{8F1B26BD-60D7-53DB-E053-6C04A8C03649},158950,2019-05-31 00:00,TS17 5FF,S,Y,F,42,NaN,CARRAWBURGH ROAD,INGLEBY BARWICK,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A


# Most expensive houses by county

This function that will take price paid data and return another DataFrame containing the
full details of the largest transaction occurring within each county present in the data.

In [47]:
def max_price_grp(df):
  # grouping by County and finding the max price
  grp_county = df.groupby('County')['Price'].max().reset_index()

  return grp_county

In [48]:
max_price_per_county = max_price_grp(price_paid_data)
max_price_per_county.head()

,County,Price
0,BATH AND NORTH EAST SOMERSET,139030009
1,BEDFORD,22457255
2,BLACKBURN WITH DARWEN,25000000
3,BLACKPOOL,5000000
4,BLAENAU GWENT,3475000


# Top 5 districts by quarterly transaction value

This function that will take price paid data and return a DataFrame (indexed by quarter)
giving the 5 postcode districts (i.e. AB1 2CD => AB1) with the largest total transaction value for
each quarter (and these values).

In [90]:
def max_price_quarter(df):

  df=df.copy()
  # to work only with dates (day)
  df.DateOfTrasfer = pd.to_datetime(df.DateOfTrasfer)
  # defining a column containing quarters Q1, Q2, Q3, Q4
  df['quarter'] = pd.PeriodIndex(df.DateOfTrasfer, freq='Q').astype(str).str[-2:]
  # grouping and calculating max value
  df = df.groupby(['quarter', 'District', 'Postcode'])['Price'].max()
  # grouping quarter and district, returing the top 5 
  df = df.groupby(level=[0,1]).head(5).reset_index()
  # sorting the dataframe
  df.sort_values(['quarter', 'District'], ascending=(True, False))
  # split the postcode by space and returns the first part of the postcode
  df['Postcode'] = df.Postcode.str.split(' ', 0, expand=True)

  df.set_index('quarter', inplace=True)

  return df

In [93]:
max_price_per_quarter = max_price_quarter(price_paid_data)
max_price_per_quarter

,District,Postcode,Price
quarter,,,
Q1,ADUR,BN14,342000
Q1,ADUR,BN15,290000
Q1,ADUR,BN15,533500
Q1,ADUR,BN15,192500
Q1,ADUR,BN15,270000
...,...,...,...
Q4,YORK,DL7,355000
Q4,YORK,YO1,255000
Q4,YORK,YO1,270000


# Transaction value concentration

This function that will take price paid data and return a DataFrame, indexed by year and
with one column for each property type, giving the percentage of transactions (in descending
order of size) that account for 80% of the total transaction value occurring for that property type
for each year.

In [80]:
def trans_value_conc(df):

  df=df.copy()
  # to work only with dates (day)
  df.DateOfTrasfer = pd.to_datetime(df.DateOfTrasfer)
  # defining a column containing the year
  df['year'] = pd.PeriodIndex(df.DateOfTrasfer, freq='A')
  # format the values in the dataframe
  pd.options.display.float_format = '£{:,.2f}'.format
  # create a new column for each property type
  # calculates the 80% of the paid price 
  df['pt_D'] = np.where(df['Prop_Type']=='D', df['Price']*0.8, 0)
  df['pt_S'] = np.where(df['Prop_Type']=='S', df['Price']*0.8, 0)
  df['pt_T'] = np.where(df['Prop_Type']=='T', df['Price']*0.8, 0)
  df['pt_F'] = np.where(df['Prop_Type']=='F', df['Price']*0.8, 0)
  df['pt_O'] = np.where(df['Prop_Type']=='O', df['Price']*0.8, 0)

  # df.drop('Prop_Type', axis=1)

  df = df.set_index('year')
  # grouping by index and calculating the total transaction value for each type
  df = df.groupby(df.index).agg({'pt_D':sum, 'pt_S':sum, 'pt_T':sum, 'pt_F':sum, 'pt_O':sum})

  return df

In [81]:
trans_sum = trans_value_conc(price_paid_data)
trans_sum

,pt_D,pt_S,pt_T,pt_F,pt_O
year,,,,,
2019,"£75,706,616,738.40","£54,278,671,798.40","£50,679,192,069.60","£41,355,572,252.80","£57,356,379,532.00"
2020,"£56,653,368,791.20","£40,107,438,449.60","£38,452,272,940.00","£24,671,770,388.00","£35,935,940,321.60"


# Volume & median price comparisons
This function will take two subsets of price paid data and returns a DataFrame
showing the percentage change in the number of transactions and their median price between
the two datasets, broken down by each of the following price brackets:

*   £0 < x <= £250,000
*   £250,000 < x <= £500,000
*   £500,000 < x <= £750,000
*   £750,000 < x <= £1,000,000
*   £1,000,000 < x <= £2,000,000
*   £2,000,000 < x <= £5,000,000
*   £5,000,000+

The return value should be a DataFrame, indexed by price bracket expressed as a 2-tuple, and
with columns for % change in transaction volume & % change in median price.

Since only two subsets are required, the function is not adaptable if more subset are needed.

In [57]:
def volume_median_price(df):

  # creating the two sets
  df_1 = df[0]
  df_2 = df[1]
  # price ranges
  bins = [0,250000,500000,750000,1000000,2000000,5000000, np.inf]
  # labels for index
  labels=[(0, 250000),(250000, 500000),(500000, 750000),
          (750000, 1000000), (1000000, 2000000),
          (2000000, 5000000), (5000000,)]
  # grouping and slicing by price ranges
  # calculating the total transactions and median price  
  df_1 = df_1.groupby(pd.cut(df_1['Price'], bins=bins, labels=labels)).agg({'Postcode':'size', 'Price':'median'})\
  .rename(columns={'Postcode':'count_1','Price':'Price_Median_1'})

  df_2 = df_2.groupby(pd.cut(df_2['Price'], bins=bins, labels=labels)).agg({'PPD':'size', 'Price':'median'})\
  .rename(columns={'PPD':'count_2','Price':'Price_Median_2'})
  # concataneting into one dataframe
  df = pd.concat([df_1, df_2], axis=1)
  # calculating count and median price percentage between the subsets
  df['count_pct_change'] = (df.count_1 - df.count_2) / df.count_1
  df['Median_Pct_Change'] = (df.Price_Median_1 - df.Price_Median_2) / df.Price_Median_1
  
  cols = [0,1,2,3]
  df.drop(df.columns[cols],axis=1,inplace=True)

  df = df.style.format({
    'count_pct_change': '{:.2%}',
    'Median_Pct_Change': '{:.2%}'}).set_properties(**{'text-align': 'center'})

  return df


In [82]:
vol_price_median = volume_median_price(all_dfs)
vol_price_median

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)


,count_pct_change,Median_Pct_Change
Price,,
"(0, 250000)",35.43%,-0.96%
"(250000, 500000)",30.79%,-0.90%
"(500000, 750000)",24.13%,0.00%
"(750000, 1000000)",21.00%,0.00%
"(1000000, 2000000)",20.56%,-1.57%
"(2000000, 5000000)",24.24%,0.00%
"(5000000,)",30.04%,11.53%
